In [82]:
import pandas as pd
import numpy as np
import regex as re
from nltk.corpus import stopwords
import unicodedata
import string
import sys
import os
import matplotlib
import math
import heapq
import cPickle as pickle
from collections import defaultdict
from HTMLParser import HTMLParser
from sklearn.cross_validation import train_test_split
from eutils.utils.logger import logger

In [2]:
html_parser = HTMLParser()

In [3]:
pd.set_option('display.max_colwidth', 500)

### Load Data

In [63]:
df = pd.read_csv('../data/output/title_category_keep_samp_small.csv')

In [64]:
# Exclude categories where count < 3
cat_count = df[['title', 'category_path']].groupby('category_path').count().reset_index().sort_values(by='title')

# Filter those with > 10 categories
cat_count = cat_count[cat_count['title'] > 5]
cat_count.rename(columns={'title': 'cat_count'}, inplace=True)

# Filter df to only include those with cat_count > 5
df = df.merge(cat_count, on='category_path', how='inner')

# Keep only necessary columns
df = df[['asin', 'title', 'category_path']]

logger.info('Data loaded of size: {}'.format(df.shape))

2016-06-17 11:28:48,693 - Data loaded of size: (102741, 3)
INFO:__log__:Data loaded of size: (102741, 3)


### Process data

In [65]:
STOP_WORDS = set(stopwords.words('english'))
SPAM_WORDS = {'import', 'export', 'day', 'week', 'month', 'year', 'new', 'free', 'international', 'intl', 'oem', ''}
COLOURS = set(matplotlib.colors.cnames.keys())
STOP_WORDS = STOP_WORDS.union(SPAM_WORDS).union(COLOURS)

In [66]:
# Remove records with no category form df
def remove_no_category(df, category='category_path'):
    """ (DataFrame, str) -> DataFrame

    Returns a dataframe where the missing categories have been dropped.

    :param df:
    :param category:
    :return:
    """

    df = df[df[category] != np.nan]
    return df


# Function to encode string
def encode_string(title, parser):
    """ (str) -> str

    Returns a string that is encoded as ascii
    Note: While unicode(title, 'utf-8', 'ignore') seems to work correctly in doctest, it has led to errors in the past.
    If so, use iso-8859-1.

    :param title:
    :return:

    >>> encode_string('Crème brûlée')
    'Creme brulee'
    >>> encode_string('åöûëî')
    'aouei'
    """

    try:
        encoded_title = unicodedata.normalize('NFKD', unicode(title, 'utf-8', 'ignore')).encode('ascii', 'ignore')
        encoded_title = parser.unescape(encoded_title).encode('ascii', 'ignore')
    except TypeError:  # if title is missing and a float
        encoded_title = 'NA'

    return encoded_title


# Encode titles in df
def encode_title(df, title='title_processed', parser=html_parser):
    """ (DataFrame, str) -> DataFrame

    Returns a dataframe where the title has been encoded.

    :param df:
    :param title:
    :return:
    """

    df[title] = df[title].apply(encode_string, args=(parser, ))
    logger.info('{} encoded'.format(title))
    return df


# Lowercase titles in df
def lowercase_title(df, title='title_processed'):
    """ (DataFrame, str) -> DataFrame

    Returns a dataframe where the title has been lowercased.

    :param df:
    :param title:
    :return:
    """

    df[title] = df[title].apply(string.lower)
    logger.info('{} lowercased'.format(title))
    return df


# Tokenize strings
def tokenize_title_string(title, excluded):
    """ (str) -> list(str)

    Returns a list of string tokens given a string.
    It will exclude the following characters from the tokenization: - / . %

    :param title:
    :return:

    >>> tokenize_title_string('hello world')
    ['hello', 'world']
    >>> tokenize_title_string('test hyphen-word 0.9 20% green/blue')
    ['test', 'hyphen-word', '0.9', '20%', 'green/blue']
    """

    return re.split("[^" + excluded + "\w]+", title)


# Tokenize titles in df
def tokenize_title(df, title='title', excluded='-/.%'):
    """ (DataFrame, str) -> DataFrame

    Returns a dataframe where the title has been tokenized based on function tokenize_title_string

    :param df:
    :param title:
    :return:
    """

    df[title] = df[title].apply(tokenize_title_string, args=(excluded, ))
    logger.info('{} tokenized'.format(title))
    return df


# Remove stopwords from string
def remove_words(title, words_to_remove):
    """ (list(str), set) -> list(str)

    Returns a list of tokens where the stopwords/spam words/colours have been removed

    :param title:
    :param words_to_remove:
    :return:
    >>> remove_words(['python', 'is', 'the', 'best'], STOP_WORDS)
    ['python', 'best']
    >>> remove_words(['grapes', 'come', 'in', 'purple', 'and', 'green'], STOP_WORDS)
    ['grapes', 'come']
    >>> remove_words(['spammy', 'title', 'intl', 'buyincoins', 'export'], STOP_WORDS)
    ['spammy', 'title']
    """

    return [token for token in title if token not in words_to_remove]


# Remove stopwords from df
def remove_stopwords(df, stopwords, title='title_processed'):
    """ (DataFrame, set, str) -> DataFrame

    Returns a DataFrame where the stopwords have been removed from the titles

    :param df:
    :param stopwords:
    :param title:
    :return:
    """
    df[title] = df[title].apply(remove_words, args=(stopwords, ))
    logger.info('{} stopwords removed'.format(title))
    return df


# Remove words with character count below threshold from string
def remove_chars(title, word_len=1):
    """ (list(str), int) -> list(str)

    Returns a list of str (tokenized titles) where tokens of character length =< word_len is removed.

    :param title:
    :param word_len:
    :return:

    >>> remove_chars(['what', 'remains', 'of', 'a', 'word', '!', ''], 1)
    ['what', 'remains', 'of', 'word']
    >>> remove_chars(['what', 'remains', 'of', 'a', 'word', '!', '', 'if', 'word_len', 'is', '2'], 2)
    ['what', 'remains', 'word', 'word_len']
    """

    return [token for token in title if len(token) > word_len]


# Remove words that are fully numeric
def remove_numeric(title):
    """ (list(str)) -> list(str)

    Remove words which are fully numeric

    :param title:
    :return:

    >>> remove_numeric(['A', 'B', '1', '123', 'C'])
    ['A', 'B', 'C']
    """

    return [token for token in title if not token.isdigit()]


# Remove words that are solely numeric from df
def remove_numeric_from_df(df, title='title_processed'):
    df[title] = df[title].apply(remove_numeric)
    logger.info('{} solely numeric words removed'.format(title))
    return df


# Remove words that have words == 1 char from title
def remove_one_char_words(df, word_len=1, title='title_processed'):
    """ (DataFrame, int, str) -> DataFrame

    Returns a DataFrame where tokens of character length <= word_len is removed

    :param df:
    :param word_len:
    :param title:
    :return:
    """

    df[title] = df[title].apply(remove_chars, args=(word_len, ))
    logger.info('{} tokens with char length equals {} removed'.format(title, word_len))
    return df

In [67]:
def find_ngrams(input_list, n):
    """ list, int -> list(tuples)

    Return a list of ngram tuples, where each tuple contains n unigrams

    :param input_list:
    :param n:
    :return:
    >>> find_ngrams(['A', 'B', 'C', 'D'], 2)
    [('A', 'B'), ('B', 'C'), ('C', 'D')]
    >>> find_ngrams(['A', 'B', 'C', 'D'], 3)
    [('A', 'B', 'C'), ('B', 'C', 'D')]
    """
    return zip(*[input_list[i:] for i in range(n)])


def create_ngram_from_tokens(tokens):
    """ list(str) -> list(str)

    Returns a list of ngram strings from a list of ngram tokens

    :param tokens:
    :return:
    >>> create_ngram_from_tokens(['A', 'B', 'C', 'D'])
    ['A', 'B', 'C', 'D', 'A_B', 'B_C', 'C_D', 'A_B_C', 'B_C_D']
    """

    bigram_list = find_ngrams(tokens, 2)
    trigram_list = find_ngrams(tokens, 3)

    bigrams = [tuple[0] + '_' + tuple[1] for tuple in bigram_list]
    trigrams = [tuple[0] + '_' + tuple[1] + '_' + tuple[2] for tuple in trigram_list]

    ngram_list = tokens + bigrams + trigrams
    return ngram_list


def create_ngram(df, title='title'):
    """ (DataFrame) -> DataFrame

    Returns a DataFrame where the title is converted from str to ngrams

    :param df:
    :param title:
    :return:
    """

    df[title] = df[title].apply(create_ngram_from_tokens)
    logger.info('{} ngrams created'.format(title))
    return df


def create_tfidf_dict(train, title='title', category='regional_key'):
    """ (DataFrame, str, str) -> defaultdict

    Returns a tf-idf dict given a dataframe containing title and regional_key

    :param train:
    :param title:
    :param category:
    :return:
    """

    # Create tf dictionary (though the name is tfidf, it's only tf for now)
    ngram_dict_tfidf = defaultdict()

    # For each token in the titles, create a dict as its value
    for i, row in train.iterrows():
        tokens = row[title]
        for token in tokens:
            ngram_dict_tfidf[token] = defaultdict()

    logger.info('TF dict phase 1 done')

    # For each token in the titles, add the token frequency to the value of the token key
    # Token frequency = token count / total number of tokens in title
    for i, row in train.iterrows():
        tokens = row[title]
        regional_id = row[category]
        for token in tokens:
            token_tf = tokens.count(token) / float(len(tokens))
            try:
                ngram_dict_tfidf[token][regional_id] += token_tf
            except KeyError:
                ngram_dict_tfidf[token][regional_id] = token_tf

    logger.info('TF dict phase 2 done')

    # create idf dictionary and count the number of titles in train
    ngram_dict_idf = defaultdict()
    no_of_skus = len(train)

    # For each token in the title, add one to the value of the token key
    for i, row in train.iterrows():
        tokens = set(row[title])
        for token in tokens:
            try:
                ngram_dict_idf[token] += 1
            except KeyError:
                ngram_dict_idf[token] = 1

    # For each token in idf dict, divide the total number of skus (logged) by the count of token value
    # Add 1 to the numerator to prevent zero divison error
    for term, count in ngram_dict_idf.iteritems():
        ngram_dict_idf[term] = math.log(no_of_skus) / float(1 + count)

    logger.info('IDF dict done')

    # Multiple values in tf dictionary with idf dictionary to get tf-idf dictionary
    for ngram, cat_dict in ngram_dict_tfidf.iteritems():
        # print ngram
        ngram_idf = ngram_dict_idf[ngram]
        # print ngram_idf
        for regional_key, count in cat_dict.iteritems():
            ngram_dict_tfidf[ngram][regional_key] = count * ngram_idf

    logger.info('TF-IDF dict done')
    return ngram_dict_tfidf

In [68]:
def merge_dicts(dicts, defaultdict=defaultdict, int=int):
    """ (list(dict), type, type) -> dict

    Returns a single dictionary given a list of dictionaries.
    Values with the same keys are summed and assigned to the key.

    :param dicts:
    :param defaultdict:
    :param int:
    :return:

    >>> merge_dicts([{'A': 1}, {'B': 2}])
    defaultdict(<type 'int'>, {'A': 1, 'B': 2})
    >>> merge_dicts([{'A': 1}, {'B': 2}, {'C': 3}, {'A': 10}])
    defaultdict(<type 'int'>, {'A': 11, 'C': 3, 'B': 2})
    """

    merged = defaultdict(int)
    for d in dicts:
        for k in d:
            merged[k] += d[k]

    return merged


def get_score(tokens, ngram_dict, top_n):
    dict_list = []

    # get list of dictionaries based on tokens
    for token in tokens:
        try:
            dict_list.append(ngram_dict[token])
        except KeyError:
            pass

    # Merge list of dicts together and add values
    score = merge_dicts(dict_list)

    # Get top n regional ids based on score
    top_n_cats = heapq.nlargest(top_n, score, key=score.get)

    return top_n_cats


def get_top_n_score(scores, n):
    try:
        n_score = scores[n - 1]
    except IndexError:
        n_score = -1

    return n_score


def create_options(df, title, tfidf_dict):
    df['options'] = df.loc[:, title].apply(get_score, args=(tfidf_dict, 3, ))
    logger.info('Test set scored')
    
    df['option1'] = df.loc[:, 'options'].apply(get_top_n_score, args=(1, ))
    df['option2'] = df.loc[:, 'options'].apply(get_top_n_score, args=(2, ))
    df['option3'] = df.loc[:, 'options'].apply(get_top_n_score, args=(3, ))
    logger.info('Top 3 options created')
    return df

In [69]:
def validate_accuracy(df):
    
    df['option1_match'] = df['category_path'] == df['option1']
    df['option2_match'] = df['category_path'] == df['option2']
    df['option3_match'] = df['category_path'] == df['option3']
    
    score1 = df['option1_match'].sum() / float(len(df))
    score2 = df['option2_match'].sum() / float(len(df))
    score3 = df['option3_match'].sum() / float(len(df))
    score123 = score1 + score2 + score3
    
    print "Scores: {}, {}, {} ({})".format(score1, score2, score3, score123)
    return df

In [70]:
# # Process titles
df = encode_title(df, title='title') 
df = lowercase_title(df, title='title')
df = tokenize_title(df, title='title', excluded='-.')
df = remove_stopwords(df, stopwords=STOP_WORDS, title='title')
df = remove_numeric_from_df(df, title='title')
df = remove_one_char_words(df, word_len=1, title='title')
df = create_ngram(df, title='title')
logger.info('Data prepped')

2016-06-17 11:28:50,992 - title encoded
INFO:__log__:title encoded
2016-06-17 11:28:51,127 - title lowercased
INFO:__log__:title lowercased
2016-06-17 11:28:51,955 - title tokenized
INFO:__log__:title tokenized
2016-06-17 11:28:52,319 - title stopwords removed
INFO:__log__:title stopwords removed
2016-06-17 11:28:52,622 - title solely numeric words removed
INFO:__log__:title solely numeric words removed
2016-06-17 11:28:52,940 - title tokens with char length equals 1 removed
INFO:__log__:title tokens with char length equals 1 removed
2016-06-17 11:28:54,068 - title ngrams created
INFO:__log__:title ngrams created
2016-06-17 11:28:54,069 - Data prepped
INFO:__log__:Data prepped


### Create path_to_int dict

In [46]:
# Get array of category_paths
category_paths = df['category_path'].unique()

In [48]:
# Sort category paths
category_paths.sort()

In [50]:
# Create category_to_int_dict
category_to_int_dict = {key: value for key, value in zip(category_paths, range(len(category_paths)))}

In [52]:
# Create int_to_category_dict
int_to_category_dict = {value: key for key, value in category_to_int_dict.items()}

### Convert category_path to int

In [72]:
# Convert category_path_to_int
df['category_path'] = df['category_path'].apply(lambda x: category_to_int_dict[x])

,asin,title,category_path
0,B0057DPWMG,"[puma, men, 3pack, volume, boxer, brief, puma_men, men_3pack, 3pack_volume, volume_boxer, boxer_brief, puma_men_3pack, men_3pack_volume, 3pack_volume_boxer, volume_boxer_brief]",2610
1,B001KWFBYC,"[underworks, mens, ultra, light, cotton, spandex, compression, tank, underworks_mens, mens_ultra, ultra_light, light_cotton, cotton_spandex, spandex_compression, compression_tank, underworks_mens_ultra, mens_ultra_light, ultra_light_cotton, light_cotton_spandex, cotton_spandex_compression, spandex_compression_tank]",2610
2,B00HY3MPZO,"[men, g-cup, string, boxer, shorts, men_g-cup, g-cup_string, string_boxer, boxer_shorts, men_g-cup_string, g-cup_string_boxer, string_boxer_shorts]",2610
3,B006VYNME0,"[balance, mens, performance, sport, brief, nb40901hb, royal, 25563-x-large, balance_mens, mens_performance, performance_sport, sport_brief, brief_nb40901hb, nb40901hb_royal, royal_25563-x-large, balance_mens_performance, mens_performance_sport, performance_sport_brief, sport_brief_nb40901hb, brief_nb40901hb_royal, nb40901hb_royal_25563-x-large]",2610
4,B002U0KKXK,"[puma, men, general, fitness, boxer, brief, blk, wht, large, puma_men, men_general, general_fitness, fitness_boxer, boxer_brief, brief_blk, blk_wht, wht_large, puma_men_general, men_general_fitness, general_fitness_boxer, fitness_boxer_brief, boxer_brief_blk, brief_blk_wht, blk_wht_large]",2610
5,B008FFCXU2,"[underworks, mens, extreme, gynecomastia, chest, binder, v-tank, top, underworks_mens, mens_extreme, extreme_gynecomastia, gynecomastia_chest, chest_binder, binder_v-tank, v-tank_top, underworks_mens_extreme, mens_extreme_gynecomastia, extreme_gynecomastia_chest, gynecomastia_chest_binder, chest_binder_v-tank, binder_v-tank_top]",2610
6,B00E8IFBZ4,"[ist, men, sliq, mesh, jock, strap, ist_men, men_sliq, sliq_mesh, mesh_jock, jock_strap, ist_men_sliq, men_sliq_mesh, sliq_mesh_jock, mesh_jock_strap]",2610
7,B001KWFC0A,"[underworks, mens, ultra, light, cotton, spandex, compression, crew, neck, t-shirt, underworks_mens, mens_ultra, ultra_light, light_cotton, cotton_spandex, spandex_compression, compression_crew, crew_neck, neck_t-shirt, underworks_mens_ultra, mens_ultra_light, ultra_light_cotton, light_cotton_spandex, cotton_spandex_compression, spandex_compression_crew, compression_crew_neck, crew_neck_t-shirt]",2610
8,B00IO6YCWY,"[mens, puma, sporty, low, rise, briefs, pack, mens_puma, puma_sporty, sporty_low, low_rise, rise_briefs, briefs_pack, mens_puma_sporty, puma_sporty_low, sporty_low_rise, low_rise_briefs, rise_briefs_pack]",2610
9,B008UDGCGK,"[champion, men, tech, performance, short, boxer, brief, champion, men, tech, champion_men, men_tech, tech_performance, performance_short, short_boxer, boxer_brief, brief_champion, champion_men, men_tech, champion_men_tech, men_tech_performance, tech_performance_short, performance_short_boxer, short_boxer_brief, boxer_brief_champion, brief_champion_men, champion_men_tech]",2610


### Create dictionary

In [75]:
tfidf_dict = create_tfidf_dict(train=df, title='title', category='category_path')

2016-06-17 11:29:25,579 - TF dict phase 1 done
INFO:__log__:TF dict phase 1 done
2016-06-17 11:29:37,362 - TF dict phase 2 done
INFO:__log__:TF dict phase 2 done
2016-06-17 11:29:45,747 - IDF dict done
INFO:__log__:IDF dict done
2016-06-17 11:29:47,048 - TF-IDF dict done
INFO:__log__:TF-IDF dict done


### Save dictionary

In [83]:
def save_dict(tfidf_dict, int_to_category_dict, output_dir, output_name):
    """ (defaultdict, str, str) -> NoneType

    Saves the dictionary (tfidf_dict) into pickle format

    :param tfidf_dict:
    :param output_dir:
    :param output_name:
    :return:
    """
    output_dir_path = os.path.join(output_dir, output_name + '.pickle')

    with open(output_dir_path, 'wb') as handle:
        pickle.dump((tfidf_dict, int_to_category_dict), handle, protocol=2)
        logger.info('Dict saved in {}'.format(output_dir_path))

In [84]:
save_dict(tfidf_dict, int_to_category_dict, './', 'test')

2016-06-17 11:35:02,039 - Dict saved in ./test.pickle
INFO:__log__:Dict saved in ./test.pickle


In [85]:
def load_dict(dict_dir='categorize', dict_name='tfidf_dict'):
    """ (str, str) -> defaultdict

    Loads a dictionary for categorization into memory

    :param tfidf_dict:
    :param dict_dir:
    :param dict_name:
    :return:
    """
    output_dir_path = os.path.join(dict_dir, dict_name + '.pickle')

    with open(output_dir_path, 'rb') as handle:
        logger.info('Dictionary loading from: {}/{}.pickle'.format(dict_dir, dict_name))
        return pickle.load(handle)

In [89]:
tf, int_to_cat = load_dict('../categorize/', 'tfidf_dict')

2016-06-17 11:50:29,542 - Dictionary loading from: ../categorize//tfidf_dict.pickle
INFO:__log__:Dictionary loading from: ../categorize//tfidf_dict.pickle
